<a href="https://colab.research.google.com/github/vanadhisivakumar-source/Machine-learning-projects/blob/main/candidate%20algorithm%20test%20case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Analyze a medical diagnosis dataset to predict disease based on symptoms. This involves loading the dataset (which should contain symptom attributes and a target variable indicating disease presence/absence), preprocessing the data (cleaning, handling missing values, encoding categorical symptoms, and splitting into training/testing sets), training a classification model (e.g., Logistic Regression, Decision Tree, Random Forest) to predict 'disease' status, evaluating the model's performance using metrics such as accuracy, precision, recall, and F1-score, displaying a confusion matrix, and visualizing the results (e.g., a plot of the classification report or a ROC curve with legends). Finally, summarize the model's ability to predict disease from symptoms, including key performance indicators and potential insights.

## Load Dataset

### Subtask:
Load the medical diagnosis dataset. The dataset should contain symptom attributes and a target variable indicating the presence or absence of the disease.


**Reasoning**:
First, I'll import the pandas library to enable data manipulation, then load the dataset from a CSV file into a DataFrame, and display the first few rows and the DataFrame's information to verify it loaded correctly.



In [ ]:
import pandas as pd

# Placeholder for dataset path. Please replace 'your_dataset.csv' with the actual path to your dataset.
df = pd.read_csv('your_dataset.csv')

print("First 5 rows of the dataset:")
print(df.head())

print("\nDataFrame Info:")
df.info()

**Reasoning**:
The previous code failed because 'your_dataset.csv' was not found. To resolve this and allow the code to execute, I will replace the placeholder with an available sample dataset (`/content/sample_data/california_housing_train.csv`) and add a comment instructing the user to replace it with their actual medical diagnosis dataset for the task.



In [ ]:
import pandas as pd

# Placeholder for dataset path. PLEASE REPLACE '/content/sample_data/california_housing_train.csv'
# with the actual path to your medical diagnosis dataset (symptom attributes and target variable for disease presence/absence).
df = pd.read_csv('/content/sample_data/california_housing_train.csv')

print("First 5 rows of the dataset:")
print(df.head())

print("\nDataFrame Info:")
df.info()

## Load Dataset

### Subtask:
Load the medical diagnosis dataset. The dataset should contain symptom attributes and a target variable indicating the presence or absence of the disease.


### Important: Dataset Replacement Required

The previous step successfully loaded a sample dataset (`california_housing_train.csv`) to demonstrate the `pd.read_csv` function. However, for this task, you need to replace `/content/sample_data/california_housing_train.csv` in the code block above with the actual path to your medical diagnosis dataset.

**Your medical diagnosis dataset should:**
- Be a CSV file.
- Contain various columns representing different symptom attributes (e.g., 'fever', 'cough', 'headache', 'fatigue').
- Include a target column indicating the presence or absence of the disease (e.g., 'disease_present', 'diagnosis', 'label'), which should ideally be a binary (0/1) or categorical variable.

Once you have updated the path to your dataset and it contains the appropriate columns, rerun the code cell. The subsequent steps will assume `df` contains a medical diagnosis dataset as described.

## Preprocess Data

### Subtask:
Clean the dataset, handle missing values, encode categorical symptom attributes if necessary, and split the data into training and testing sets.


**Reasoning**:
The first step in data preprocessing is to inspect the data types and check for missing values to understand the dataset's structure and identify any immediate issues that need addressing.



In [ ]:
print("DataFrame Info:")
df.info()

print("\nMissing values per column:")
print(df.isnull().sum())

**Reasoning**:
Based on the instructions, the next step involves handling missing values, encoding categorical features, and splitting the data into training and testing sets. I will generate code that includes these steps, providing general methods for preprocessing, and specifically reminding the user to update the target column name, as the current DataFrame is a placeholder.



In [ ]:
print("Starting data preprocessing...")

# 2. Handle Missing Values
# For demonstration, a general approach is shown. Please adjust based on your actual medical dataset's characteristics.
print("\nHandling missing values (if any)...")
initial_missing_count = df.isnull().sum().sum()
if initial_missing_count > 0:
    print(f"Initial total missing values: {initial_missing_count}")
    # Impute numerical columns with mean and categorical columns with mode
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype == 'object' or df[col].dtype == 'category':
                df[col].fillna(df[col].mode()[0], inplace=True)
                print(f"  Filled missing values in categorical column '{col}' with mode.")
            else:
                df[col].fillna(df[col].mean(), inplace=True)
                print(f"  Filled missing values in numerical column '{col}' with mean.")
else:
    print("No missing values found in the DataFrame.")
print(f"Total missing values after handling: {df.isnull().sum().sum()}")

# 3. Identify and Encode Categorical Features
print("\nEncoding categorical features (if any)...")
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

if len(categorical_cols) > 0:
    print(f"Categorical columns identified: {list(categorical_cols)}")
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    print("Categorical features encoded using one-hot encoding.")
else:
    print("No categorical columns of type 'object' or 'category' found for encoding.")
    print("If your medical dataset has categorical symptoms, ensure their dtype is 'object' or 'category' or explicitly list them for encoding.")

# 4. Define Features and Target, then Split Data
print("\nDefining features and target, and splitting data...")

# !!! IMPORTANT: Replace 'disease_status' with the actual name of your target column !!!
# This column should indicate the presence or absence of the disease.
# For example, if your target column is named 'diagnosis', change 'disease_status' to 'diagnosis'.
target_column = 'disease_status'

# Verify if the target column exists in the DataFrame
if target_column not in df.columns:
    raise KeyError(f"Target column '{target_column}' not found in the DataFrame. Please update the 'target_column' variable to the actual name of your disease status column from your medical dataset.")

X = df.drop(columns=[target_column])
y = df[target_column]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nShape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

print("\nData preprocessing steps complete.")

**Reasoning**:
The previous code failed because the target column 'disease_status' was not found in the placeholder DataFrame `df` (california_housing_train.csv). To fix this, I will create a new binary column from 'median_house_value' to simulate a disease presence/absence target, which will allow the code to run and align with the overall classification task, while explicitly instructing the user to replace this with their actual medical diagnosis target later.



In [ ]:
print("Starting data preprocessing...")

# 2. Handle Missing Values
# For demonstration, a general approach is shown. Please adjust based on your actual medical dataset's characteristics.
print("\nHandling missing values (if any)...")
initial_missing_count = df.isnull().sum().sum()
if initial_missing_count > 0:
    print(f"Initial total missing values: {initial_missing_count}")
    # Impute numerical columns with mean and categorical columns with mode
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype == 'object' or df[col].dtype == 'category':
                df[col].fillna(df[col].mode()[0], inplace=True)
                print(f"  Filled missing values in categorical column '{col}' with mode.")
            else:
                df[col].fillna(df[col].mean(), inplace=True)
                print(f"  Filled missing values in numerical column '{col}' with mean.")
else:
    print("No missing values found in the DataFrame.")
print(f"Total missing values after handling: {df.isnull().sum().sum()}")

# 3. Identify and Encode Categorical Features
print("\nEncoding categorical features (if any)...")
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

if len(categorical_cols) > 0:
    print(f"Categorical columns identified: {list(categorical_cols)}")
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    print("Categorical features encoded using one-hot encoding.")
else:
    print("No categorical columns of type 'object' or 'category' found for encoding.")
    print("If your medical dataset has categorical symptoms, ensure their dtype is 'object' or 'category' or explicitly list them for encoding.")

# 4. Define Features and Target, then Split Data
print("\nDefining features and target, and splitting data...")

# !!! IMPORTANT: For the actual medical diagnosis task, replace this section to
# define your target column based on the presence/absence of disease.
# The current placeholder dataset (california_housing_train.csv) does not have a 'disease_status' column.
# We will create a *simulated* binary target for demonstration purposes using 'median_house_value'.
# You MUST replace this logic with your actual disease target column when using your medical dataset.

# Create a binary target from 'median_house_value' for demonstration
# This simulates a binary classification problem (e.g., disease present/absent)
median_value_threshold = df['median_house_value'].median()
df['simulated_disease_status'] = (df['median_house_value'] > median_value_threshold).astype(int)

target_column = 'simulated_disease_status' # Set the new simulated target column

# Now, ensure to drop the original 'median_house_value' if it's the source of the simulated target
# to prevent data leakage and to make the features truly independent.
# For this example, we drop 'median_house_value' as 'simulated_disease_status' is derived from it.
if 'median_house_value' in df.columns:
    df = df.drop(columns=['median_house_value'])

# The original error check for target_column is no longer needed as we've defined a valid one.

X = df.drop(columns=[target_column])
y = df[target_column]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nShape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

print("\nData preprocessing steps complete.")

## Train Disease Prediction Model

### Subtask:
Train a classification model (e.g., Logistic Regression, Decision Tree, Random Forest) using the preprocessed symptom data to predict the 'disease' status.


**Reasoning**:
I will import the `LogisticRegression` model, instantiate it with a `random_state` for reproducibility, and then train the model using the preprocessed `X_train` and `y_train` datasets.



In [ ]:
print("Training a classification model...")

# 1. Choose a classification model: Logistic Regression
from sklearn.linear_model import LogisticRegression

# 2. Instantiate the model
# Using random_state for reproducibility
model = LogisticRegression(random_state=42, solver='liblinear') # solver='liblinear' is often robust for small datasets and binary classification

# 3. Train the model
model.fit(X_train, y_train)

print("Model training complete. The Logistic Regression model has been trained.")

## Evaluate Model Performance

### Subtask:
Evaluate the trained model's performance using appropriate metrics such as accuracy, precision, recall, F1-score, and display a confusion matrix. Visualize the results, including a plot of the classification report or a ROC curve if applicable, with legends.


**Reasoning**:
I will make predictions on the test set, calculate standard classification metrics (accuracy, precision, recall, F1-score), generate a confusion matrix, and visualize the classification report to evaluate the model's performance.



In [ ]:
print("Evaluating model performance...")

# 1. Make predictions on the test set
y_pred = model.predict(X_test)

# 2. Calculate and print accuracy score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy:.4f}")

# 3. Calculate precision, recall, and F1-score using classification_report
print("\nClassification Report:")
class_report = classification_report(y_test, y_pred, output_dict=True)
print(classification_report(y_test, y_pred))

# 4. Generate and display a confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# 5. Visualize the classification report
# Using ClassificationReportDisplay if available or print the dictionary as alternative
from sklearn.metrics import ClassificationReportDisplay

# Prepare the classification report dictionary for display if it's not already in the right format
# ClassificationReportDisplay typically takes the result of classification_report(output_dict=True)
# or can generate it itself. We'll use the already calculated class_report dictionary.

# ClassificationReportDisplay often works best with raw y_true and y_pred if you want it to compute
# and format. If you have the dictionary, you might print it or visualize manually.

# Let's use the direct method with plot_classification_report if available or simulate it.
# A simple way to visualize the report directly is by printing the output of classification_report.
# For a more visual representation, let's plot the precision, recall, f1-score.

# Get class names from y_test unique values, ensure sorted order
class_names = [str(x) for x in sorted(y_test.unique())]

# If ClassificationReportDisplay is not directly plotting, we can create a text representation plot
# Or, rely on the print of classification_report. For visual output, we can create a bar plot.

# Option 1: Use ClassificationReportDisplay if it's meant for plotting directly
# As of scikit-learn v1.0+, ClassificationReportDisplay can be used to plot
display = ClassificationReportDisplay.from_predictions(y_test, y_pred, cmap='viridis')
display.plot()
plt.title('Classification Report Visualization')
plt.show()

print("Model evaluation complete. Results displayed.")

**Reasoning**:
The previous code failed because `ClassificationReportDisplay` could not be imported, indicating a potential version incompatibility or missing feature in the installed scikit-learn. To fix this, I will remove the problematic import and instead add a visualization of the ROC curve, which is another effective way to evaluate classification model performance and was also mentioned in the subtask instructions.



In [ ]:
print("Evaluating model performance...")

# 1. Make predictions on the test set
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1] # Probabilities for the positive class

# 2. Calculate and print accuracy score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy:.4f}")

# 3. Calculate precision, recall, and F1-score using classification_report
print("\nClassification Report:")
class_report = classification_report(y_test, y_pred, output_dict=True)
print(classification_report(y_test, y_pred))

# 4. Generate and display a confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# 5. Plot ROC curve
print("\nROC Curve:")
# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print("Model evaluation complete. Results displayed.")

## Final Task

### Subtask:
Summarize the model's ability to predict disease based on symptoms, including key performance indicators and potential insights.


## Summary:

### Q&A
The model, a Logistic Regression, demonstrated a reasonable ability to predict the simulated disease status from the California Housing dataset features. Key performance indicators included an accuracy of approximately 0.8035 and an AUC (Area Under the Curve) of 0.80. The precision, recall, and F1-score for both classes were consistently around 0.80. These metrics suggest that the model can distinguish between the two classes with moderate success on this particular dataset.

### Data Analysis Key Findings
*   The initial dataset loading step required user intervention to replace a placeholder with an actual medical diagnosis dataset. The analysis proceeded using the `california_housing_train.csv` dataset, with a simulated target variable.
*   During data preprocessing, no missing values or categorical features were identified in the placeholder `california_housing_train.csv` dataset.
*   A simulated binary target variable, `simulated_disease_status`, was created from the `median_house_value` column to enable classification, as a true disease status column was absent in the placeholder dataset.
*   The data was successfully split into training and testing sets, with `X_train` having a shape of (13600, 8) and `X_test` (3400, 8).
*   A Logistic Regression model was trained using the preprocessed data.
*   The trained model achieved an accuracy of approximately 0.8035 on the test set.
*   The classification report showed balanced performance for both classes, with precision, recall, and F1-score all around 0.80.
*   The ROC curve indicated an AUC (Area Under the Curve) of 0.80, suggesting a moderate ability to discriminate between the two classes in the simulated problem.

### Insights or Next Steps
*   The current model performance is based on a simulated disease prediction using the California Housing dataset. The insights gained are thus limited to this specific, non-medical context.
*   To obtain meaningful results for disease prediction, the analysis must be rerun with an actual medical diagnosis dataset as specified in the task description. This is the most crucial next step to validate the model's true predictive capability for disease based on symptoms.
